# Spark RDD-Dataframe Solution

<font color='steelblue'>
<span style="font-family:Arial; font-size:1.6em;">
    <b>Number of practice problems have been defined using a computer usage dataset</b><br>
    <ol>
        <li>Data is to be read from a text file into RDD <b>(../datasets/LoginData.txt)</b></li>
        <li>Convert it to dataframe</li>
        <li>Practice problemsn are marked in Red</li>
    </ol>    
</span>
</font>

## Set Up Spark Environment

In [ ]:
# Set up the environment for using pyspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

In [ ]:
# Create Application Context
spark = SparkSession.builder.appName("Spark Assignment").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("Error")

<font color='tomato'>
    <h2>Read the text file "LoginData.txt"</h2><br>
<span style="font-family:Arial; font-size:1.4em;">
    Do the following on the text file:
    <ol>
        <li>Use option <b>use_unicode=True</b> while reading text file into a RDD</li>
        <li>Replace the " (double quotes) with '' (empty string)</li>
        <li>Separate the data based on , (comma)</li>
        <li>Display 2 rows in RDD</li>
    </ol>
    </span>
</font>

In [ ]:
aRDD = sc.textFile("../datasets/LoginData.txt", use_unicode=True) \
                    .map(lambda x:x.replace('"', "")) \
                    .map(lambda x:x.split(","))

In [ ]:
# In my environment have to do a cache otherwise the code crashes
aRDD.cache()
aRDD.take(2)

<font color='tomato'>
    <h2>Convert the RDD into Dataframe</h2><br>
<span style="font-family:Arial; font-size:1.4em;">
    <b>Note the first item in the RDD is the column heading</b><br>
    Will have to implement the following:
    <ol>
        <li>Ignore the first item in RDD and use it as the columns</li>
        <li>Let spark interpret the schema from the data</li>
        <li>Display column names</li>
        <li>Show first 5 rows of dataframe</li>
        <li>Display the datatypes</li>
         <li>Print the number of rows and number of columns in the dataframe</li>
    </ol>
    </span>
</font>

In [ ]:
df = spark.createDataFrame(data = aRDD.filter(lambda x:x[0] != 'date'),
                        schema=aRDD.filter(lambda x:x[0] == 'date').\
                        collect()[0])

In [ ]:
df.columns

In [ ]:
# persist the dataframe since it will be used a lot
df.persist()

In [ ]:
df.show(5)

In [ ]:
df.dtypes

In [ ]:
print('Rows: {} Columns: {}'.format(df.count(), len(df.columns)))

In [ ]:
df.describe().show(truncate = False)

<font color='tomato'>
    <h2>Data type changes</h2><br>
<span style="font-family:Arial; font-size:1.4em;">
    Implement the following:
    <ol>
        <li>Convert the date column from string type to date type</li>
        <li>Convert the size column from string to integer</li>
        <li>Show first 5 rows of dataframe</li>
        <li>Display the datatypes</li>
    </ol>
    </span>
</font>

In [ ]:
from pyspark.sql.types import IntegerType, DateType

In [ ]:
df = df.withColumn("date", df["date"].cast(DateType()))

In [ ]:
df = df.withColumn("size", df["size"].cast(IntegerType()))

In [ ]:
df.show(5)

In [ ]:
df.dtypes

<font color='tomato'>
    <h2>Data Exploration</h2><br>
<span style="font-family:Arial; font-size:1.4em;">
    Implement the following:
    <ol>
        <li>Sort by size column in ascending order and display 10 rows</li>
        <li>Sort by size column in descending order and display 10 rows</li>
        <li>Count Null, NaN or NA values in r_version column</li>
        <li>Count Null, or NA values in all columns</li>
        <li>Get count by country</li>
        <li>Get unique machine architectures in r_arch column</li>
        <li>Get unique combinations machine architectures & OS in r_arch and r_os columns</li>
    </ol>
    </span>
</font>

In [ ]:
df.sort(df.size.asc()).show(10)

In [ ]:
df.sort(df.size.desc()).show(10)

In [ ]:
from pyspark.sql.functions import isnan, when, count, col, sum

In [ ]:
# NaN, NULL or NA values in r_version column
df.select([count(when(isnan('r_version') | 
                      col('r_version').isNull() | 
                      col('r_version').contains('NA'), True))]).show()

In [ ]:
# Count Null or NA values in all columns
df.select([count(when(col(c).isNull() | 
                      col(c).contains('NA'), c)).alias(c) 
           for c in df.columns]).show()

In [ ]:
# Get count by country
df.groupBy('country').count().show(truncate = False)

In [ ]:
# Unique machine architectures
df.select('r_arch').distinct().collect()

In [ ]:
# Unique combination of architecture and os
df.select(['r_arch', 'r_os']).distinct().collect()

<font color='tomato'>
    <h2>Filtering and Aggregation</h2><br>
<span style="font-family:Arial; font-size:1.4em;">
    Implement the following:
    <ol>
        <li>Get percentage of records in the size column where the values is less than 1000</li>
        <li>Get percentage of ggplot2 packages used in package column</li>
        <li>Count the packages per type and show the top 20</li>
        <li><b>BONUS:</b> Create percentage for each package using user defined function</li>
    </ol>
    </span>
</font>

In [ ]:
# Get percentage of size data that is less than 1000
df.filter(df['size'] < 1000).count() / df.count()

In [ ]:
# Unique packages
df.select('package').distinct().count()

In [ ]:
# Get the percentage of ggplot2 package used
df.filter(df['package'] == "ggplot2").count() / df.count()

In [ ]:
package_count = df.groupBy("package").count().sort("count", ascending = False)
package_count.show()

In [ ]:
# Create a user defined function (UDF)
from pyspark.sql.functions import udf

pkgPerc = udf(lambda x: str(round(x * 100, 3)) + "%")

In [ ]:
package_count = package_count.withColumn("percentage", pkgPerc(package_count['count'] / df.count()))
package_count.show()